In [1]:
import pandas as pd
from haversine import haversine
import numpy as np

# 데이터 불러오기

In [2]:
# DB 연결
from DB import *
db_connection = connect_db()

## 메인데이터 xy좌표 불러오기 (main)

In [3]:
main = load_data_from_rds('생태통로', db_connection)

## 교통량 데이터 불러오기 (traffic)

In [4]:
traffic= pd.read_excel('.././data/variables/2021년 도로 종류별 교통량 및 X.Y좌표.xlsx', sheet_name = 'X.Y좌표')

## 환경성 영향평가 데이터 불러오기 (env) 

In [5]:
env = pd.read_csv('.././data/variables/국토환경성 평가지도 병합.csv', encoding = 'cp949')

## 건물까지거리 데이터 불러오기 (build)

In [6]:
build = pd.read_csv(".././data/variables/건물까지거리_병합.csv", encoding = 'cp949')

# 1. 교통량 전처리

## 7.8km 이내에 교통량 찾기 

In [7]:
count = 0
index = []
for i in range(len(main)):
    for j in range(len(traffic)):
        dist = haversine((main[:]["위도"][i],main[:]["경도"][i]), (traffic[:]["XCODE"][j],traffic[:]["YCODE"][j]), unit = 'km')
        count +=1
        if dist <= 7.82 : #각 구간간 거리평균이 7.82
            index.append([i,j]) # 7.8km 이내에 있는 교통량의 인덱스 리스트에 넣기

In [8]:
# 리스트 데이터 프레임으로 만들기
index2 = pd.DataFrame(index) # 0 : 생태통로 index, 1: 교통량 index
index2

,0,1
0,0,1180
1,0,1182
2,0,1183
3,0,1649
4,0,1651
...,...,...
4338,496,1743
4339,496,2535
4340,496,2538
4341,496,2590


In [9]:
# 교통량 값만 불러오기
AADT=pd.DataFrame(traffic["AADT"].iloc[index2[1]])
AADT= AADT.reset_index()

In [14]:
# 교통량 값이랑 인덱스 데이터 프레임이랑 합치기 
index3 = pd.concat([index2, AADT],axis =1)

# 하나의 생태통로에 여러개의 교통량이 관측되면 평균값으로 계산
index4 = index3[[0,"AADT"]].groupby(0).mean()
index4

,AADT
0,
0,7493.875000
1,3573.857143
2,3430.428571
3,3430.428571
4,34270.190476
...,...
492,34270.190476
493,34542.400000
494,34270.190476


In [15]:
prepro_1 = pd.concat([main['번호'], index4], axis = 1)
prepro_1

,번호,AADT
0,1495,7493.875000
1,1277,3573.857143
2,1278,3430.428571
3,1279,3430.428571
4,1090,34270.190476
...,...,...
492,1089,34270.190476
493,1087,34542.400000
494,1088,34270.190476
495,1091,26748.437500


# 2. 환경 영향성 평가 전처리

In [16]:
prepro_2 = pd.merge(prepro_1, env, left_on = "번호", right_on = "번호", how = "inner")[['번호','AADT','SAMPLE_1']]

In [17]:
prepro_2

,번호,AADT,SAMPLE_1
0,1495,7493.875000,2
1,1277,3573.857143,1
2,1278,3430.428571,2
3,1279,3430.428571,2
4,1090,34270.190476,5
...,...,...,...
492,1089,34270.190476,5
493,1087,34542.400000,5
494,1088,34270.190476,5
495,1091,26748.437500,2


# 3. 건물까지 거리 전처리

In [18]:
# m 단위 km 단위로 바꾸기
build['distance_km'] = build['distance']/1000

In [19]:
prepro_3 = pd.merge(prepro_2, build, left_on = "번호", right_on = "번호", how = "inner")[['번호','AADT', 'SAMPLE_1', 'distance_km']]

# 열 이름 바꾸기

In [20]:
prepro_3.columns = ['번호', '교통량', '환경영향평가점수', '건물까지거리(km)']
prepro_3

,번호,교통량,환경영향평가점수,건물까지거리(km)
0,1495,7493.875000,2,0.196597
1,1277,3573.857143,1,0.358139
2,1278,3430.428571,2,0.139054
3,1279,3430.428571,2,0.044559
4,1090,34270.190476,5,0.172184
...,...,...,...,...
492,1089,34270.190476,5,0.179017
493,1087,34542.400000,5,0.094304
494,1088,34270.190476,5,0.124789
495,1091,26748.437500,2,0.070303


# 데이터 베이스에 올리기

## 변수 데이터 불러오기

In [21]:
variables = load_data_from_rds('변수관계설정', db_connection)

In [22]:
# 변수 데이터랑 합치기
final = pd.merge(variables , prepro_3, on = '번호')

In [23]:
final

,번호,경사도,농가까지의 거리(km),최고제한속도(km/h),등산로까지 최단거리(km),식생,주변 로드킬 빈도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,하천거리(km),산책로까지의 최단 거리(km),교통량,환경영향평가점수,건물까지거리(km)
0,1001,6,0.125686,80.0,1.903039,0,0,X,NaN,NaN,1594.082919,237.040073,173030.400000,1,0.192281
1,1002,5,0.074627,60.0,0.120319,1,0,X,4.0,4.0,1434.181414,122.055124,55985.500000,1,0.575364
2,1004,2,0.110617,60.0,0.047211,1,0,산지,1.0,1.0,1248.795538,125.516218,46452.800000,2,0.077290
3,1005,2,0.009030,80.0,0.233820,1,0,X,4.0,4.0,937.738738,128.701343,36727.714286,4,0.155399
4,1006,4,0.098269,80.0,2.172428,0,0,하천,3.0,3.0,215.408967,103.572133,107833.500000,1,0.104287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1561,2,0.087448,60.0,0.000416,1,4,산지,3.0,5.0,987.166229,219.634740,107659.473684,1,0.271655
493,1562,5,0.137822,80.0,2.835246,1,0,산지,6.0,27.0,1164.042354,38.432601,13853.200000,2,0.252010
494,1564,4,0.054073,80.0,0.886170,1,0,X,1.0,1.0,931.149822,248.086515,6478.333333,2,0.113175
495,1567,5,0.102655,60.0,9.125829,1,0,하천,4.0,5.0,1002.913219,283.374921,10937.272727,2,0.248477


In [24]:
# DB에 올리기
upload_data_to_rds(final, '변수관계설정', db_connection)